In [21]:
import pandas as pd
import numpy as np

#读取文件
sojournTime_raw = pd.read_csv('sojournTime.csv',index_col=0)
comDelay_raw    = pd.read_csv('comDelay.csv',index_col=0)
load_raw     = pd.read_csv('load.csv',index_col=0)
memoryUtil_raw  = pd.read_csv('memoryUtil.csv',index_col=0)

# 得到cloudlet数量
cloudlet_numbers = sojournTime_raw.shape[1]

# 对comDelay和load进行同向化
comDelay_txh = comDelay_raw.apply(lambda x: (np.max(x) - x)) 
load_txh = load_raw.apply(lambda x: (np.max(x) - x)) 

# 合并成一个DataFrame
data = pd.concat([sojournTime_raw,comDelay_txh,load_txh,memoryUtil_raw],axis=1)

# 对所有数据进行归一化
data_nor = data.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
data_nor


,sojournTime0,sojournTime1,sojournTime2,sojournTime3,sojournTime4,sojournTime5,sojournTime6,sojournTime7,sojournTime8,sojournTime9,...,memoryUtil10,memoryUtil11,memoryUtil12,memoryUtil13,memoryUtil14,memoryUtil15,memoryUtil16,memoryUtil17,memoryUtil18,memoryUtil19
T0,0.000000,0.220823,0.211634,0.409481,0.294586,0.143260,0.385737,0.054680,0.014521,0.984873,...,0.457143,0.966667,0.058824,0.617647,0.000000,0.769231,0.12,0.107143,0.552632,0.911765
T1,0.091234,1.000000,0.000000,0.717630,0.177389,0.270423,0.171799,0.652943,0.866893,0.252389,...,0.942857,0.333333,0.000000,0.970588,0.277778,0.192308,1.00,0.714286,0.052632,1.000000
T2,1.000000,0.471592,0.070914,0.832889,0.000000,0.000000,0.629822,0.608877,1.000000,1.000000,...,0.742857,0.766667,0.911765,0.000000,0.694444,0.076923,0.56,0.535714,0.026316,0.294118
T3,0.843713,0.186958,1.000000,0.366222,1.000000,0.445070,0.946191,0.000000,0.059051,0.183121,...,0.971429,0.133333,0.029412,0.352941,0.916667,0.692308,0.00,0.107143,0.000000,0.294118
T4,0.307418,0.207669,0.985042,1.000000,0.303185,0.664386,0.000000,0.467674,0.171346,0.000000,...,0.342857,0.066667,1.000000,0.323529,0.777778,0.230769,0.48,1.000000,0.236842,0.794118
T5,0.362555,0.390708,0.549030,0.000000,0.539172,1.000000,1.000000,0.525893,0.934656,0.377389,...,1.000000,0.833333,0.088235,1.000000,0.611111,0.653846,0.48,0.607143,0.000000,0.676471
T6,0.032130,0.685139,0.988920,0.589037,0.357006,0.609658,0.688817,1.000000,0.612294,0.294586,...,0.000000,0.233333,0.441176,0.029412,0.527778,1.000000,0.04,0.500000,0.000000,0.617647
T7,0.259024,0.000000,0.972853,0.231407,0.580573,0.280080,0.099514,0.241878,0.708132,0.535828,...,0.800000,1.000000,0.794118,0.382353,0.472222,0.000000,0.64,0.535714,0.973684,0.000000
T8,0.598969,0.439407,0.263158,0.143407,0.335987,0.553320,0.725446,0.598263,0.000000,0.148487,...,0.485714,0.233333,0.294118,0.029412,0.583333,0.192308,0.44,0.642857,1.000000,0.970588
T9,0.382388,0.308144,0.971745,0.548444,0.331847,0.821328,0.705024,0.373110,0.324298,0.400478,...,0.428571,0.000000,0.794118,0.352941,1.000000,0.000000,0.12,0.000000,0.973684,0.088235


In [22]:
# 云模型处理
def cloud_model(df):
    Ex = df.mean()
    En = np.sqrt(np.pi/2)  * np.mean( np.abs(df-Ex) )
    He = np.sqrt(np.abs(df.mean() * df.mean() - df.std() * df.std()))
    return pd.concat([Ex,En,He],axis=1)
data_cloudModel = cloud_model(data_nor)
data_cloudModel

,0,1,2
sojournTime0,0.387743,0.320711,0.193595
sojournTime1,0.391044,0.258674,0.267924
sojournTime2,0.601330,0.479245,0.423063
sojournTime3,0.483852,0.318026,0.367541
sojournTime4,0.391975,0.236581,0.284963
...,...,...,...
memoryUtil15,0.380769,0.399132,0.116934
memoryUtil16,0.388000,0.318843,0.224186
memoryUtil17,0.475000,0.303481,0.357125
memoryUtil18,0.381579,0.494729,0.234434


In [23]:
# 中智聚合

# 中智数
truth = data_cloudModel[0]
indeterminacy = data_cloudModel[1]
falsity = data_cloudModel[2]

# 权重
weight = [0.25,0.25,0.25,0.25]

# 聚合之后的中智数
aggre_truth = []
aggre_indeterminacy = []
aggre_falsity = []

for i in range(cloudlet_numbers):
    tempT = 1
    tempI = 1
    tempF = 1
    # 四个属性
    for j in range(4):
        tempT *= (1 - truth[i+10*j]) ** weight[j]
        tempI *= indeterminacy[i+10*j] ** weight[j]
        tempF *= falsity[i+10*j] ** weight[j]
    aggre_truth.append((1-tempT))
    aggre_indeterminacy.append(tempI)
    aggre_falsity.append(tempF)

# 添加索引
# index = pd.Series(np.arange(0,cloudlet_numbers))
# index = index.astype(str)
# index = 'cloudlet'+index
neutrosophic = pd.DataFrame(np.vstack((aggre_truth,aggre_indeterminacy,aggre_falsity)).T)
neutrosophic


,0,1,2
0,0.468767,0.305705,0.322828
1,0.484575,0.292182,0.352018
2,0.571099,0.373038,0.436926
3,0.476705,0.366530,0.310509
4,0.412808,0.259121,0.254990
5,0.492543,0.336593,0.345442
6,0.418035,0.360352,0.143293
7,0.493556,0.320051,0.363307
8,0.503184,0.370967,0.322020
9,0.476993,0.318540,0.303974


In [24]:
# 得分函数
score = neutrosophic[0]+1-neutrosophic[1]+1-neutrosophic[2]
print(score)
print('最佳cloudlet为：',score.idxmax())

with open("optimal_cloudlet.txt","w") as f:
        f.write(str(score.idxmax()))  #这句话自带文件关闭功能，不需要再写f.close()


0     1.840234
1     1.840375
2     1.761135
3     1.799666
4     1.898696
5     1.810508
6     1.914389
7     1.810199
8     1.810197
9     1.854479
10    1.813622
11    1.811238
12    1.908074
13    1.802038
14    1.896401
15    1.825492
16    1.918700
17    1.818029
18    1.807520
19    1.842484
dtype: float64
最佳cloudlet为： 16
